In [28]:
import pandas as pd

#98 days of data for each county
df = pd.read_csv("confirmed-covid-19-cases-in-us-by-state-and-county.csv") 
df = df[df.county_name != "Statewide Unallocated"]
df.dropna(inplace = True)
df = df.reset_index()
df.head()

,index,county_fips,county_name,state_name,state_fips,date,confirmed,lat,long,geometry
0,98,1001,Autauga County,AL,1,2020-01-22,0,32.539527,-86.644082,POINT (-86.64408227 32.53952745)
1,99,1001,Autauga County,AL,1,2020-01-23,0,32.539527,-86.644082,POINT (-86.64408227 32.53952745)
2,100,1001,Autauga County,AL,1,2020-01-24,0,32.539527,-86.644082,POINT (-86.64408227 32.53952745)
3,101,1001,Autauga County,AL,1,2020-01-25,0,32.539527,-86.644082,POINT (-86.64408227 32.53952745)
4,102,1001,Autauga County,AL,1,2020-01-26,0,32.539527,-86.644082,POINT (-86.64408227 32.53952745)


### Plot Total Case Distribution Across Counties by Date

In [29]:
import folium
from folium import plugins

Marker Clustering

In [32]:
data = df
data = data[["date", "lat", "long", "county_name", "county_fips", "confirmed"]]
####DATE####
date = "2020-03-15"
data = data[data.date == date]

usa_map = folium.Map(
    location=[data["lat"].mean(), 
    data["long"].mean()], 
    zoom_start=4)

cases = plugins.MarkerCluster().add_to(usa_map)

for lat, long, confirmed in zip(data.lat, data.long, data.confirmed):
    for case in range(0, confirmed):
        folium.Marker(
            location=[lat, long],
            icon=None,
        ).add_to(cases)

usa_map.save("marker_cluster_usa_2020-03-15.html")
usa_map

Choropleth Map

In [36]:
import branca

data = df
data = data[["date", "lat", "long", "county_name", "county_fips", "confirmed"]]
####DATE####
date = "2020-04-28"
data = data[data.date == date]

usa_map = folium.Map(
    location=[data["lat"].mean(), 
    data["long"].mean()], 
    zoom_start=4)

colorscale = branca.colormap.linear.YlOrRd_09.scale(0, 100) #Scale 

fips_cases = data.set_index("county_fips")["confirmed"]

def style_function(feature):
    cases = fips_cases.get(int(feature['id'][-5:]), None)
    return {
        'fillOpacity': 0.5,
        'weight': 0,
        'fillColor': '#black' if cases is None else colorscale(cases)
    }

c_map = folium.Map(
    location = [data["lat"].mean(), data["long"].mean()],
    tiles = "cartodbpositron",
    zoom_start = 4
)

folium.TopoJson(
    open('us_counties_20m_topo.geojson'),
    'objects.us_counties_20m',
    style_function=style_function
).add_to(c_map)

c_map.save("2020-04-28_chloropleth.html")
c_map